## 챌린지
---
(EN)
- In a new Jupyter notebook create a research AI agent and give it custom tools.
- The agent should be able to do the following tasks:
    - Search in Wikipedia
    - Search in DuckDuckGo
    - Scrape and extract the text of any website.
    - Save the research to a .txt file
- Run the agent with this query: "Research about the XZ backdoor", the agent should try to search in Wikipedia or DuckDuckGo, if it finds a website in DuckDuckGo it should enter the website and extract it's content, then it should finish by saving the research to a .txt file.

(KR)
- 새로운 Jupyter notebook에서 리서치 AI 에이전트를 만들고 커스텀 도구를 부여합니다.
- 에이전트는 다음 작업을 수행할 수 있어야 합니다:
    - Wikipedia에서 검색
    - DuckDuckGo에서 검색
    - 웹사이트의 텍스트를 스크랩하고 추출합니다.
    - 리서치 결과를 .txt 파일에 저장하기
- 다음 쿼리로 에이전트를 실행합니다: "Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면, 에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고, DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음 .txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.


In [17]:
import openai
import wikipedia
import requests
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup

In [18]:
# Define AI agent tools
def search_wikipedia(query):
    try:
        summary = wikipedia.summary(query, sentences=5)
        return f"Wikipedia Summary:\n{summary}"
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Wikipedia Disambiguation Error: {e.options[:5]}"
    except wikipedia.exceptions.PageError:
        return "Wikipedia Page not found."

def search_duckduckgo(query):
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=5))
    if results:
        return [result["href"] for result in results if "href" in result]
    return []

def scrape_website(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            paragraphs = soup.find_all('p')
            text = '\n'.join([p.get_text() for p in paragraphs])
            return text[:2000]  # Limit extracted text for readability
        else:
            return "Failed to fetch the website."
    except Exception as e:
        return f"Error scraping website: {str(e)}"

def save_research_to_file(research_text, filename="research.txt"):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(research_text)

In [19]:
# AI Agent Execution
def research_ai_agent(query):
    research_results = []
    
    # Step 1: Search Wikipedia
    wiki_result = search_wikipedia(query)
    research_results.append(wiki_result)
    
    # Step 2: Search DuckDuckGo
    urls = search_duckduckgo(query)
    if urls:
        research_results.append("\nDuckDuckGo Found URLs:\n" + "\n".join(urls))
        
        # Step 3: Scrape first relevant URL
        scraped_text = scrape_website(urls[0])
        research_results.append("\nExtracted Website Content:\n" + scraped_text)
    
    # Step 4: Save research to file
    final_research = '\n\n'.join(research_results)
    save_research_to_file(final_research)
    
    return final_research

In [20]:
# Run the agent with the query "Research about the XZ backdoor"
query = "Research about the XZ backdoor"
research_result = research_ai_agent(query)
print(research_result)

Wikipedia Summary:
In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.


DuckDuckGo Found URLs:
https://research.swtch.com/xz-timeline
https://boehs.org/node/everything-i-know-about-the-xz-backdoor
https://en.wikipedia.org/wi